# Neuro-Symbolic AI

## RFT Dataset

### Contents

- Generate Dataset
- Perception Model
- Semantic Parser
- Program Executor


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm 
from img_utils import *
from data_generator import *

In [2]:
# Generate the dataset

# Dataset params
img_size = 224
object_size = 15 # the object size?

# Possibles Answers : [yes, no, rectangle, circle, 1, 2, 3, 4, 5, 6]
colors = [
    (0,0,255), # red
    (0,255,0), # green
    (255,0,0), # blue
    (0,156,255), # orange
    (128,128,128), # gray (128,128,128)- not working, replaced with black
    (0,255,255) # yellow - appears same as orange
]

In [3]:
n_train_imgs = 20
build_dataset(n_train_imgs, img_size, colors, object_size,  data_dir='data/train', prefix='train') 

100%|██████████| 20/20 [00:00<00:00, 28.57it/s]
